# 📊 WhatsApp Wrapped

**Create beautiful Spotify Wrapped-style visualizations for your WhatsApp group chats!**

This notebook analyzes your WhatsApp chat exports and generates stunning HTML reports with:
- 📈 Rich Analytics (message counts, activity patterns, emoji usage)
- 🎨 Beautiful Visualizations (interactive Plotly charts with dark theme)
- 👥 User Insights (top contributors, activity sparklines)
- 📅 Calendar Heatmaps (activity across the year)
- 💬 Message Analysis (word patterns, response times)

---

## 🚀 How to Use

1. **Run Step 1** - Install dependencies (only needed once)
2. **Run Step 2** - Upload your chat file and see available options
3. **Run Step 3** - Select output format (HTML or PDF)
4. **Run Step 4** - Configure filters and generate your report!

### 📱 How to Export WhatsApp Chat
1. Open WhatsApp and navigate to the group chat
2. Tap the group name → More → Export chat
3. Choose **"Without Media"** for faster processing
4. Save the `.zip` file to upload here

---

🔒 **Privacy First**: All processing happens in this notebook. Your data is never uploaded anywhere else.

In [ ]:
#@title 🔧 **Step 1: Setup** (Run this first!) { display-mode: "form" }
#@markdown This cell installs all required dependencies and clones the WhatsApp Wrapped repository.
#@markdown 
#@markdown **Click the play button ▶️ to run this cell.**

import subprocess
import sys
import os
from IPython.display import display, HTML, clear_output

# Check if packages were already installed (marker file exists)
packages_installed = os.path.exists("/content/.packages_installed")

if not packages_installed:
    print("📦 Installing dependencies...")
    print("="*50)
    
    # Install required packages (suppress most output)
    packages = [
        "pandas>=2.0.0",
        "numpy>=1.24.0",
        "matplotlib>=3.7.0",
        "seaborn>=0.12.0",
        "plotly>=5.14.0",
        "plotly-calplot>=0.1.20",
        "wordcloud>=1.9.0",
        "pillow>=10.0.0",
        "python-dateutil>=2.8.0",
        "pyyaml>=6.0",
        "tqdm>=4.65.0",
        "jinja2>=3.1.0",
        "emojis>=0.7.0",
    ]
    
    for pkg in packages:
        pkg_name = pkg.split(">=")[0].split("[")[0]
        print(f"  • Installing {pkg_name}...", end=" ")
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", "-q", pkg],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            print("✓")
        else:
            print("⚠️ (may already be installed)")
    
    # Create marker file and restart runtime
    with open("/content/.packages_installed", "w") as f:
        f.write("done")
    
    print("\n" + "="*50)
    print("⚠️  RESTARTING RUNTIME...")
    print("="*50)
    print("\n⚡ Runtime will restart to apply package updates.")
    print("👉 Please RUN THIS CELL AGAIN after restart.\n")
    
    display(HTML("""
    <div style="background: #ff9800; padding: 20px; border-radius: 10px; margin: 20px 0;
                font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
        <h3 style="color: white; margin: 0 0 10px 0;">⚡ Runtime Restarting...</h3>
        <p style="color: white; margin: 0;">Please <strong>run this cell again</strong> after the runtime restarts to complete setup.</p>
    </div>
    """))
    
    # Force runtime restart
    os.kill(os.getpid(), 9)

# Packages already installed, proceed with repo clone
print("✓ Dependencies already installed\n")
print("📥 Cloning WhatsApp Wrapped repository...")
print("="*50)

repo_path = "/content/whatsapp-wrapped"

if os.path.exists(repo_path):
    print("  • Repository already exists, updating...")
    result = subprocess.run(
        ["git", "-C", repo_path, "pull", "--quiet"],
        capture_output=True, text=True
    )
    print("  ✓ Repository updated!")
else:
    result = subprocess.run(
        ["git", "clone", "--depth", "1", 
         "https://github.com/Duelion/whatsapp-wrapped.git", repo_path],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        print("  ✓ Repository cloned successfully!")
    else:
        print(f"  ⚠️ Clone failed: {result.stderr}")

# Add to Python path
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

print("\n" + "="*50)
print("✅ Setup complete! Proceed to Step 2.")
print("="*50)

# Display styled success message
display(HTML("""
<div style="background: linear-gradient(135deg, #1DB954, #191414); 
            padding: 20px; border-radius: 10px; margin-top: 20px;
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
    <h3 style="color: white; margin: 0;">🎉 Ready to go!</h3>
    <p style="color: #b3b3b3; margin: 10px 0 0 0;">All dependencies installed. Run the next cell to configure and generate your report.</p>
</div>
"""))

In [ ]:
#@title 📤 **Step 2: Upload Chat File** { display-mode: "form" }
#@markdown Upload your WhatsApp chat export (.zip or .txt) to analyze it.
#@markdown 
#@markdown After uploading, you'll see available years and options for your chat.

from google.colab import files
from IPython.display import display, HTML, clear_output
from pathlib import Path
import tempfile
import os
import sys

print("📤 Please upload your WhatsApp chat export (.zip or .txt)...")
print("="*60)

try:
    uploaded = files.upload()
except Exception as e:
    print(f"\n⚠️ Upload cancelled or failed: {e}")
    uploaded = None

if uploaded:
    # Get the uploaded filename
    filename = list(uploaded.keys())[0]
    file_content = uploaded[filename]
    
    print(f"\n✓ Uploaded: {filename} ({len(file_content):,} bytes)")
    print("\n" + "="*60)
    print("🔍 Analyzing chat structure...")
    print("="*60 + "\n")
    
    # Save the uploaded file to a persistent location
    upload_dir = "/content/uploaded_chat"
    os.makedirs(upload_dir, exist_ok=True)
    chat_path = os.path.join(upload_dir, filename)
    with open(chat_path, 'wb') as f:
        f.write(file_content)
    
    try:
        # Import the parser
        repo_path = "/content/whatsapp-wrapped"
        if repo_path not in sys.path:
            sys.path.insert(0, repo_path)
        
        from src.parser import parse_whatsapp_export
        
        # Quick parse to extract metadata (no filtering)
        print("[1/2] 📖 Parsing chat file...")
        df_full, metadata_full = parse_whatsapp_export(
            chat_path,
            filter_system=True,
            min_messages=1,  # Include all users for analysis
            year_filter=None,  # No year filter for initial analysis
        )
        print(f"      ✓ Found {len(df_full):,} messages from {metadata_full.total_members} members")
        
        # Extract available years
        print("[2/2] 📅 Extracting available years...")
        available_years = sorted(df_full['timestamp'].dt.year.unique(), reverse=True)
        years_with_counts = df_full.groupby(df_full['timestamp'].dt.year).size().sort_index(ascending=False)
        print(f"      ✓ Found messages from {len(available_years)} years")
        
        # Store data for next cell
        # Using a simple approach: save to files that the next cell can read
        import json
        analysis_info = {
            "chat_path": chat_path,
            "filename": filename,
            "total_messages": len(df_full),
            "total_members": metadata_full.total_members,
            "available_years": [int(y) for y in available_years],
            "years_with_counts": {str(k): int(v) for k, v in years_with_counts.items()},
            "date_range_start": metadata_full.date_range_start.isoformat(),
            "date_range_end": metadata_full.date_range_end.isoformat(),
            "member_names": metadata_full.member_names,
        }
        with open("/content/chat_analysis.json", "w") as f:
            json.dump(analysis_info, f)
        
        print("\n" + "="*60)
        print("✅ CHAT ANALYZED SUCCESSFULLY!")
        print("="*60)
        
        # Build year options HTML
        year_rows = ""
        for year, count in years_with_counts.items():
            pct = (count / len(df_full)) * 100
            year_rows += f"""
                <tr>
                    <td style="padding: 6px 16px 6px 0; color: #1DB954; font-weight: 600;">{year}</td>
                    <td style="padding: 6px 16px 6px 0; color: #e0e0e0;">{count:,} messages</td>
                    <td style="padding: 6px 0; color: #999;">({pct:.1f}%)</td>
                </tr>"""
        
        # Display analysis results
        display(HTML(f"""
        <div style="background: #2a2a2a; 
                    padding: 20px; border-radius: 8px; margin: 20px 0;
                    border-left: 3px solid #1DB954;
                    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
            <div style="color: #e0e0e0; font-size: 16px; font-weight: 600; margin-bottom: 12px;">
                📊 {metadata_full.filename}
            </div>
            <div style="display: flex; gap: 20px; margin: 12px 0; color: #b3b3b3; font-size: 14px;">
                <span><strong style="color: #1DB954;">{len(df_full):,}</strong> messages</span>
                <span>•</span>
                <span><strong style="color: #1DB954;">{metadata_full.total_members}</strong> members</span>
                <span>•</span>
                <span><strong style="color: #1DB954;">{len(available_years)}</strong> years</span>
            </div>
            <div style="color: #999; font-size: 13px; margin-top: 8px;">
                {metadata_full.date_range_start.strftime('%b %d, %Y')} - {metadata_full.date_range_end.strftime('%b %d, %Y')}
            </div>
        </div>
        
        <div style="background: #2a2a2a; padding: 18px; border-radius: 8px; margin-top: 10px;">
            <div style="color: #b3b3b3; font-size: 13px; font-weight: 600; margin-bottom: 10px; text-transform: uppercase; letter-spacing: 0.5px;">
                📅 Available Years
            </div>
            <table style="color: #b3b3b3; border-collapse: collapse; width: 100%; font-size: 14px;">
                {year_rows}
            </table>
        </div>
        """))
        
    except Exception as e:
        import traceback
        print(f"\n❌ Error analyzing chat: {e}")
        print("\nFull error details:")
        traceback.print_exc()
        
        display(HTML("""
        <div style="background: #5c1c1c; padding: 20px; border-radius: 10px; margin-top: 20px;">
            <h4 style="color: #ff6b6b; margin: 0 0 10px 0;">⚠️ Troubleshooting Tips</h4>
            <ul style="color: #ffb3b3; margin: 0; padding-left: 20px;">
                <li>Make sure you uploaded a valid WhatsApp export file (.zip or .txt)</li>
                <li>The file should be exported "Without Media" from WhatsApp</li>
                <li>Try running Step 1 (Setup) again if you see import errors</li>
            </ul>
        </div>
        """))
else:
    display(HTML("""
    <div style="background: #3d3d00; padding: 20px; border-radius: 10px; margin-top: 20px;">
        <h4 style="color: #ffeb3b; margin: 0 0 10px 0;">📤 No file uploaded</h4>
        <p style="color: #fff9c4; margin: 0;">Run this cell again and upload your WhatsApp chat export file.</p>
    </div>
    """))

In [ ]:
#@title 📄 **Step 3: Select Output Format** { display-mode: "form" }
#@markdown Choose whether you want an HTML report only, or also generate a PDF.
#@markdown 
#@markdown **PDF generation requires installing Chromium (~150MB) and may take 1-2 minutes on first run.**

import json
import os
import subprocess
import sys
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Check if chat was uploaded
if not os.path.exists("/content/chat_analysis.json"):
    display(HTML("""
    <div style="background: #5c1c1c; padding: 20px; border-radius: 10px;">
        <h4 style="color: #ff6b6b; margin: 0 0 10px 0;">⚠️ No chat uploaded</h4>
        <p style="color: #ffb3b3; margin: 0;">Please run <strong>Step 2</strong> first to upload your WhatsApp chat file.</p>
    </div>
    """))
else:
    # Load analysis data for display
    with open("/content/chat_analysis.json", "r") as f:
        analysis = json.load(f)
    
    # Display chat info header
    display(HTML(f"""
    <div style="background: #282828; padding: 15px; border-radius: 8px; margin-bottom: 20px;
                font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
        <h4 style="color: #1DB954; margin: 0 0 10px 0;">📊 {analysis['filename']}</h4>
        <p style="color: #b3b3b3; margin: 0;">
            {analysis['total_messages']:,} messages • {analysis['total_members']} members
        </p>
    </div>
    """))
    
    # Format selection radio buttons
    format_radio = widgets.RadioButtons(
        options=[
            ('📄 HTML only (fast)', 'html'),
            ('📄 HTML + 📕 PDF (requires Chromium installation)', 'pdf')
        ],
        value='html',
        description='Output:',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='450px')
    )
    
    # Warning box (shown only when PDF is selected)
    warning_box = widgets.HTML(value="")
    
    def update_warning(change):
        if change['new'] == 'pdf':
            warning_box.value = """
            <div style="background: #3d3d00; padding: 15px; border-radius: 8px; margin: 10px 0;
                        border-left: 3px solid #ffeb3b;">
                <h4 style="color: #ffeb3b; margin: 0 0 8px 0;">⏱️ PDF Generation Notice</h4>
                <ul style="color: #fff9c4; margin: 0; padding-left: 20px; font-size: 14px;">
                    <li>Requires downloading Chromium browser (~150MB)</li>
                    <li>First-time setup takes <strong>1-2 minutes</strong></li>
                    <li>Subsequent runs will be faster</li>
                </ul>
            </div>
            """
        else:
            warning_box.value = ""
    
    format_radio.observe(update_warning, names='value')
    
    # Confirm button
    confirm_button = widgets.Button(
        description='✓ Confirm & Prepare',
        button_style='success',
        layout=widgets.Layout(width='180px', height='38px')
    )
    
    output_area = widgets.Output()
    
    # Display widgets
    print("📋 Select your output format:\n")
    display(widgets.VBox([
        format_radio,
        warning_box,
        widgets.HTML("<br>"),
        confirm_button,
        output_area
    ]))
    
    def confirm_format(btn):
        with output_area:
            clear_output()
            
            selected_format = format_radio.value
            
            if selected_format == 'pdf':
                print("\n" + "="*60)
                print("🔧 Preparing PDF generation environment...")
                print("="*60 + "\n")
                
                # Check if Playwright is already installed
                playwright_installed = os.path.exists("/content/.playwright_installed")
                
                if not playwright_installed:
                    print("[1/2] 📦 Installing Playwright...")
                    result = subprocess.run(
                        [sys.executable, "-m", "pip", "install", "-q", "playwright"],
                        capture_output=True, text=True
                    )
                    if result.returncode == 0:
                        print("      ✓ Playwright installed")
                    else:
                        print(f"      ⚠️ Warning: {result.stderr}")
                    
                    print("[2/2] 🌐 Installing Chromium browser (this may take 1-2 minutes)...")
                    result = subprocess.run(
                        [sys.executable, "-m", "playwright", "install", "chromium"],
                        capture_output=True, text=True
                    )
                    if result.returncode == 0:
                        print("      ✓ Chromium installed")
                        # Create marker file
                        with open("/content/.playwright_installed", "w") as f:
                            f.write("done")
                    else:
                        print(f"      ⚠️ Warning: {result.stderr}")
                        display(HTML("""
                        <div style="background: #5c1c1c; padding: 15px; border-radius: 8px; margin-top: 10px;">
                            <h4 style="color: #ff6b6b; margin: 0 0 8px 0;">⚠️ Chromium installation may have failed</h4>
                            <p style="color: #ffb3b3; margin: 0; font-size: 14px;">
                                PDF generation might not work. You can still generate HTML reports.
                            </p>
                        </div>
                        """))
                else:
                    print("✓ Playwright and Chromium already installed\n")
            
            # Save format choice
            format_info = {"output_format": selected_format}
            with open("/content/output_format.json", "w") as f:
                json.dump(format_info, f)
            
            print("\n" + "="*60)
            print("✅ FORMAT CONFIGURED!")
            print("="*60)
            
            format_label = "HTML + PDF" if selected_format == 'pdf' else "HTML only"
            display(HTML(f"""
            <div style="background: linear-gradient(135deg, #1DB954, #191414); 
                        padding: 20px; border-radius: 10px; margin-top: 20px;
                        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
                <h3 style="color: white; margin: 0 0 10px 0;">✓ Ready to generate!</h3>
                <p style="color: #b3b3b3; margin: 0;">
                    Output format: <strong style="color: #1DB954;">{format_label}</strong>
                </p>
                <p style="color: #b3b3b3; margin: 10px 0 0 0;">
                    👉 Proceed to <strong>Step 4</strong> to configure and generate your report.
                </p>
            </div>
            """))
    
    confirm_button.on_click(confirm_format)

In [ ]:
#@title 🎛️ **Step 4: Configure & Generate Report** { display-mode: "form" }
#@markdown Configure your report options and generate!
#@markdown 
#@markdown Select which users and years to include in your report.

import json
import os
import sys
from pathlib import Path
from datetime import datetime
from IPython.display import display, HTML, clear_output
from google.colab import files
import ipywidgets as widgets

# Check if chat was uploaded
if not os.path.exists("/content/chat_analysis.json"):
    display(HTML("""
    <div style="background: #5c1c1c; padding: 20px; border-radius: 10px;">
        <h4 style="color: #ff6b6b; margin: 0 0 10px 0;">⚠️ No chat uploaded</h4>
        <p style="color: #ffb3b3; margin: 0;">Please run <strong>Step 2</strong> first to upload your WhatsApp chat file.</p>
    </div>
    """))
elif not os.path.exists("/content/output_format.json"):
    display(HTML("""
    <div style="background: #5c1c1c; padding: 20px; border-radius: 10px;">
        <h4 style="color: #ff6b6b; margin: 0 0 10px 0;">⚠️ Output format not selected</h4>
        <p style="color: #ffb3b3; margin: 0;">Please run <strong>Step 3</strong> first to select your output format (HTML or PDF).</p>
    </div>
    """))
else:
    # Load analysis data
    with open("/content/chat_analysis.json", "r") as f:
        analysis = json.load(f)
    
    # Load format choice
    with open("/content/output_format.json", "r") as f:
        format_info = json.load(f)
    output_format = format_info.get("output_format", "html")
    generate_pdf = output_format == "pdf"
    
    # Build year options from actual chat data (only specific years, no "All Years")
    year_options = [str(y) for y in analysis["available_years"]]
    
    # Build user options (all selected by default)
    member_names = analysis["member_names"]
    
    # Create widgets
    year_dropdown = widgets.Dropdown(
        options=year_options,
        value=year_options[0],  # Default to most recent year
        description='Year:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='300px')
    )
    
    # User checkboxes (all selected by default)
    user_checkboxes = []
    for name in member_names:
        checkbox = widgets.Checkbox(
            value=True,
            description=name,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        user_checkboxes.append(checkbox)
    
    # Select all / Deselect all buttons
    select_all_btn = widgets.Button(
        description='Select All',
        button_style='info',
        layout=widgets.Layout(width='100px', height='28px')
    )
    deselect_all_btn = widgets.Button(
        description='Clear',
        button_style='warning',
        layout=widgets.Layout(width='100px', height='28px')
    )
    
    def select_all(btn):
        for checkbox in user_checkboxes:
            checkbox.value = True
    
    def deselect_all(btn):
        for checkbox in user_checkboxes:
            checkbox.value = False
    
    select_all_btn.on_click(select_all)
    deselect_all_btn.on_click(deselect_all)
    
    # Report name text input
    report_name_input = widgets.Text(
        value='',
        placeholder='Leave empty for default name',
        description='Report Name:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='350px')
    )
    
    generate_button = widgets.Button(
        description='🚀 Generate Report',
        button_style='success',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    output_area = widgets.Output()
    
    # Display chat info header
    format_label = "📄 HTML + 📕 PDF" if generate_pdf else "📄 HTML only"
    display(HTML(f"""
    <div style="background: #282828; padding: 15px; border-radius: 8px; margin-bottom: 20px;
                font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
        <h4 style="color: #1DB954; margin: 0 0 10px 0;">📊 {analysis['filename']}</h4>
        <p style="color: #b3b3b3; margin: 0;">
            {analysis['total_messages']:,} messages • {analysis['total_members']} members • 
            Years: {', '.join(str(y) for y in analysis['available_years'])}
        </p>
        <p style="color: #b3b3b3; margin: 8px 0 0 0; font-size: 13px;">
            Output: <strong style="color: #1DB954;">{format_label}</strong>
        </p>
    </div>
    """))
    
    # Display widgets
    print("📋 Configure your report:\n")
    
    # Create a scrollable container for checkboxes
    checkbox_container = widgets.VBox(
        user_checkboxes,
        layout=widgets.Layout(
            max_height='200px',
            overflow_y='auto',
            border='1px solid #ccc',
            padding='5px',
            width='320px'
        )
    )
    
    display(widgets.VBox([
        widgets.HBox([widgets.Label("📅", layout=widgets.Layout(width='30px')), year_dropdown]),
        widgets.HBox([
            widgets.Label("👥", layout=widgets.Layout(width='30px')), 
            widgets.VBox([
                widgets.HBox([select_all_btn, deselect_all_btn]),
                checkbox_container
            ])
        ]),
        widgets.HBox([widgets.Label("📝", layout=widgets.Layout(width='30px')), report_name_input]),
        widgets.HTML("<br>"),
        generate_button,
        output_area
    ]))
    
    def generate_report(btn):
        with output_area:
            clear_output()
            
            # Get selected users from checkboxes
            selected_users = [checkbox.description for checkbox in user_checkboxes if checkbox.value]
            
            if not selected_users:
                display(HTML("""
                <div style="background: #5c1c1c; padding: 20px; border-radius: 10px; margin-top: 10px;">
                    <h4 style="color: #ff6b6b; margin: 0 0 10px 0;">⚠️ No users selected</h4>
                    <p style="color: #ffb3b3; margin: 0;">Please select at least one user to include in the report.</p>
                </div>
                """))
                return
            
            print("\n" + "="*60)
            print("🔄 Generating your WhatsApp Wrapped report...")
            print("="*60 + "\n")
            
            try:
                # Import modules
                repo_path = "/content/whatsapp-wrapped"
                if repo_path not in sys.path:
                    sys.path.insert(0, repo_path)
                
                from src.parser import parse_whatsapp_export
                from src.analytics import analyze_chat, format_hour, get_hour_emoji
                from src.charts import ChartCollection, create_user_sparkline, create_user_hourly_sparkline, chart_to_html
                from jinja2 import Environment, FileSystemLoader
                
                # Get widget values
                year_filter = year_dropdown.value
                fixed = True  # Always use fixed layout
                
                year_value = int(year_filter)  # Always a specific year
                
                # Parse chat with filters
                print(f"[1/4] 📖 Parsing chat file (Year: {year_filter}, Users: {len(selected_users)})...")
                df, metadata = parse_whatsapp_export(
                    analysis["chat_path"],
                    filter_system=True,
                    min_messages=1,  # Include all, we'll filter by selected users
                    year_filter=year_value,
                )
                
                # Filter to selected users only
                df = df[df["name"].isin(selected_users)].copy()
                df = df.reset_index(drop=True)
                
                # Update metadata
                from src.parser import get_chat_metadata
                metadata = get_chat_metadata(df, analysis["filename"])
                
                print(f"      ✓ Found {len(df):,} messages from {metadata.total_members} members")
                
                # Run analytics
                print("[2/4] 📊 Running analytics...")
                analytics = analyze_chat(df)
                print(f"      ✓ Analyzed {analytics.total_days} days of chat history")
                
                # Generate charts
                print("[3/4] 📈 Generating visualizations...")
                chart_collection = ChartCollection(analytics)
                charts_html = chart_collection.to_html_dict(include_plotlyjs_first=True)
                
                # Generate sparklines
                user_sparklines = {}
                user_hourly_sparklines = {}
                for user_stat in analytics.user_stats[:12]:
                    sparkline_fig = create_user_sparkline(user_stat.daily_activity, user_stat.name)
                    user_sparklines[user_stat.name] = chart_to_html(sparkline_fig, include_plotlyjs=False)
                    hourly_sparkline_fig = create_user_hourly_sparkline(user_stat.hourly_activity, user_stat.name)
                    user_hourly_sparklines[user_stat.name] = chart_to_html(hourly_sparkline_fig, include_plotlyjs=False)
                print(f"      ✓ Created {len(charts_html)} charts and {len(user_sparklines)} sparklines")
                
                # Render HTML
                print("[4/4] 🎨 Rendering HTML report...")
                template_dir = Path("/content/whatsapp-wrapped/src/templates")
                env = Environment(loader=FileSystemLoader(template_dir), autoescape=False)
                template = env.get_template("report.html")
                
                formatted_hour = format_hour(analytics.most_active_hour)
                hour_emoji = get_hour_emoji(analytics.most_active_hour)
                
                html_content = template.render(
                    metadata=metadata,
                    analytics=analytics,
                    charts=charts_html,
                    user_sparklines=user_sparklines,
                    user_hourly_sparklines=user_hourly_sparklines,
                    generation_date=datetime.now().strftime("%Y-%m-%d %H:%M"),
                    fixed_layout=fixed,
                    formatted_hour=formatted_hour,
                    hour_emoji=hour_emoji,
                )
                print("      ✓ Report rendered successfully!")
                
                # Save report
                custom_name = report_name_input.value.strip()
                if custom_name:
                    # Use custom name
                    stem = custom_name.replace(" ", "_")
                    output_filename = f"{stem}_report.html"
                else:
                    # Use default naming (filename + year)
                    stem = Path(analysis["filename"]).stem.replace(" ", "_")
                    output_filename = f"{stem}_{year_value}_report.html"
                
                output_path = f"/content/{output_filename}"
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(html_content)
                
                # Generate PDF if requested
                pdf_output_path = None
                if generate_pdf:
                    print("\n[5/5] 📕 Converting to PDF...")
                    try:
                        from src.generator import generate_pdf_report
                        pdf_output_path = output_path.replace('.html', '.pdf')
                        generate_pdf_report(output_path, pdf_output_path, quiet=False)
                        print("      ✓ PDF generated successfully!")
                    except Exception as pdf_error:
                        print(f"      ⚠️ PDF generation failed: {pdf_error}")
                        print("      HTML report is still available.")
                        pdf_output_path = None
                
                print("\n" + "="*60)
                print("🎉 REPORT GENERATED SUCCESSFULLY!")
                print("="*60)
                
                # Display summary
                display(HTML(f"""
                <div style="background: linear-gradient(135deg, #1DB954, #191414); 
                            padding: 25px; border-radius: 12px; margin: 20px 0;
                            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
                    <h2 style="color: white; margin: 0 0 15px 0;">📊 {metadata.filename}</h2>
                    <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px;">
                        <div style="background: rgba(0,0,0,0.3); padding: 15px; border-radius: 8px; text-align: center;">
                            <div style="color: #1DB954; font-size: 28px; font-weight: bold;">{analytics.total_messages:,}</div>
                            <div style="color: #b3b3b3; font-size: 12px;">MESSAGES</div>
                        </div>
                        <div style="background: rgba(0,0,0,0.3); padding: 15px; border-radius: 8px; text-align: center;">
                            <div style="color: #1DB954; font-size: 28px; font-weight: bold;">{analytics.total_members}</div>
                            <div style="color: #b3b3b3; font-size: 12px;">MEMBERS</div>
                        </div>
                        <div style="background: rgba(0,0,0,0.3); padding: 15px; border-radius: 8px; text-align: center;">
                            <div style="color: #1DB954; font-size: 28px; font-weight: bold;">{analytics.total_days}</div>
                            <div style="color: #b3b3b3; font-size: 12px;">DAYS</div>
                        </div>
                    </div>
                    <div style="margin-top: 15px; padding-top: 15px; border-top: 1px solid rgba(255,255,255,0.1);">
                        <span style="color: #b3b3b3;">Date Range:</span>
                        <span style="color: white;">{metadata.date_range_start.strftime('%b %d, %Y')} - {metadata.date_range_end.strftime('%b %d, %Y')}</span>
                    </div>
                </div>
                """))
                
                # Download
                print("\n📥 Downloading your report...")
                files.download(output_path)
                
                if pdf_output_path:
                    import time
                    time.sleep(1)  # Small delay between downloads
                    files.download(pdf_output_path)
                
                # Build "What's Next?" content based on output format
                if pdf_output_path:
                    whats_next_content = """
                    <ul style="color: #b3b3b3; margin: 0; padding-left: 20px;">
                        <li>Open the <strong>PDF file</strong> to view or share your report</li>
                        <li>Open the <strong>HTML file</strong> in a browser for interactive charts</li>
                        <li>Share the report with your group members</li>
                        <li>Generate another report with different year/filters!</li>
                    </ul>
                    """
                else:
                    whats_next_content = """
                    <ul style="color: #b3b3b3; margin: 0; padding-left: 20px;">
                        <li>Open the downloaded HTML file in any web browser</li>
                        <li>Share the report with your group members</li>
                        <li>Print to PDF from your browser for a permanent copy</li>
                        <li>Generate another report with different year/filters!</li>
                    </ul>
                    """
                
                display(HTML(f"""
                <div style="background: #282828; padding: 20px; border-radius: 10px; margin-top: 20px;
                            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
                    <h4 style="color: #1DB954; margin: 0 0 10px 0;">💡 What's Next?</h4>
                    {whats_next_content}
                </div>
                """))
                
            except Exception as e:
                import traceback
                print(f"\n❌ Error generating report: {e}")
                print("\nFull error details:")
                traceback.print_exc()
    
    generate_button.on_click(generate_report)

---

<div style="background: linear-gradient(135deg, #282828, #121212); padding: 30px; border-radius: 15px; margin: 30px 0; font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
  
### 🔒 Privacy & Security

<div style="background: rgba(29, 185, 84, 0.1); border-left: 4px solid #1DB954; padding: 15px; border-radius: 8px; margin: 15px 0;">
  
✓ **All processing happens locally** in this Colab notebook  
✓ **Your chat data is never stored** or uploaded to any server  
✓ **The generated report** is saved only to your browser's downloads  
✓ **Open source & transparent** - audit the code yourself

</div>

---

<div style="text-align: center; padding: 20px 0;">
  <h3 style="color: #1DB954; margin-bottom: 10px;">Made with ❤️ for WhatsApp users who love data</h3>
  <p style="color: #b3b3b3; margin-bottom: 25px;">Created by <a href="https://github.com/Duelion" style="color: #1DB954; text-decoration: none;">@Duelion</a></p>
  
  <div style="display: flex; gap: 15px; justify-content: center; align-items: center; flex-wrap: wrap;">
    <a href="https://github.com/Duelion/whatsapp-wrapped" target="_blank" style="text-decoration: none;">
      <div style="background: #1DB954; color: white; padding: 12px 24px; border-radius: 25px; font-weight: bold; display: inline-flex; align-items: center; gap: 8px; transition: transform 0.2s;">
        ⭐ Star on GitHub
      </div>
    </a>
    <a href="https://buymeacoffee.com/duelion" target="_blank" style="text-decoration: none;">
      <div style="background: #FFDD00; color: #000; padding: 12px 24px; border-radius: 25px; font-weight: bold; display: inline-flex; align-items: center; gap: 8px;">
        ☕ Buy Me a Coffee
      </div>
    </a>
  </div>
  
  <p style="color: #666; margin-top: 25px; font-size: 12px;">
    💡 <strong>Enjoying WhatsApp Wrapped?</strong> Share it with your friends and star the repo!
  </p>
</div>

</div>